In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import requests
import json
import geopy

import time
import parquet


In [2]:
data = pd.read_csv('despesas-entre-2003-e-2022.csv', encoding='latin1', sep=';', dtype='str')

In [3]:
data['SUBELEMENTO DE DESPESA'] = data['SUBELEMENTO DE DESPESA'].astype('category')

In [4]:
data

,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC
0,02/01/2003,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,"R$ 1.696,90",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N
1,02/01/2003,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,"R$ 1.336,60",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N
2,02/01/2003,***.868.251-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,"R$ 1.396,43",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N
3,03/01/2003,***.004.131-**,00592717000170,FLORES ALVORADA,"R$ 8.585,00",C,OUTROS SERVIOS DE TERCEIROS-PESSOA JURÖDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N
4,03/01/2003,***.004.131-**,00839308000125,CRISTAL LIMPEZA - COMRCIO DE MATERIAIS PARA L...,"R$ 251,50",C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N
...,...,...,...,...,...,...,...,...
113337,04/12/2022,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIÇO,"518,07",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N
113338,04/12/2022,***.136.591-**,12550441000146,RESTAURANTE SOHO,"330,52",C,FORNECIMENTO DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N
113339,04/12/2022,***.136.591-**,03696869000100,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"54,66",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N
113340,Fonte: SUPRIM,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
def get_company_info(cnpj, use_safe_access=True):
    url = f"https://receitaws.com.br/v1/cnpj/{cnpj}"
    querystring = {"token":"XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX","cnpj":"06990590000123","plugin":"RF"}
    if use_safe_access:
        time.sleep(20)
    response = requests.request("GET", url, params=querystring)
    if use_safe_access and not response.ok:
        time.sleep(60)
        response = requests.request("GET", url, params=querystring)
    return json.loads(response.text)

def get_company_adress(info):
    return f"{info['logradouro']}, {info['numero']}, {info['municipio']}, {info['uf']}"

In [6]:
GOOGLE_CNPJ = "03696869000100"
info = get_company_info(GOOGLE_CNPJ, use_safe_access=False)
print(info)
#atividade_principal

#tipo

#logradouro
#número
#bairro
#municipio
#uf
#cep

#natureza_juridica


{'atividade_principal': [{'code': '47.11-3-02', 'text': 'Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - supermercados'}], 'data_situacao': '03/11/2005', 'complemento': 'BLOCO A', 'tipo': 'MATRIZ', 'nome': 'BIG TRANS COMERCIAL DE ALIMENTOS S/A', 'abertura': '04/11/1999', 'telefone': '(61) 3265-191', 'qsa': [{'nome': 'JORGE HELOU FILHO', 'qual': '10-Diretor'}, {'nome': 'MARIO HABKA', 'qual': '10-Diretor'}], 'situacao': 'ATIVA', 'bairro': 'ASA NORTE', 'logradouro': 'SHCN EQ 402/403', 'numero': 'SN', 'cep': '70.834-400', 'municipio': 'BRASILIA', 'uf': 'DF', 'porte': 'DEMAIS', 'natureza_juridica': '205-4 - Sociedade Anônima Fechada', 'fantasia': 'BIG BOX SUPERMERCADOS', 'cnpj': '03.696.869/0001-00', 'ultima_atualizacao': '2022-12-27T17:17:28.815Z', 'status': 'OK', 'email': '', 'efr': '', 'motivo_situacao': '', 'situacao_especial': '', 'data_situacao_especial': '', 'atividades_secundarias': [{'code': '00.00-0-00', 'text': 'Não informada'}], 'capital_

In [7]:
data['ATIVIDADE_PRINCIPAL'] = ''
data['MATRIZ'] = ''
data['NATUREZA_JURIDICA'] = ''
data['LOGRADOURO'] = ''
data['NUMERO'] = ''
data['COMPLEMENTO'] = ''
data['BAIRRO'] = ''
data['MUNICIPIO'] = ''
data['UF'] = ''
data['CEP'] = ''

In [3]:
data = pd.read_csv('Dados_processados_metade.csv', encoding='latin1', dtype='str')

info_dict = {}
line = 286
while len(info_dict) < 900:
    cnpj = data['CPF/CNPJ FORNECEDOR'].iloc[line]
    info = None
    if cnpj in info_dict:
        info = info_dict[cnpj]
    else:
        try:
            info = get_company_info(cnpj)
            info_dict[cnpj] = info
        except:
            print(f"Erro: linha {line}")
    try:
        data.loc[line,'ATIVIDADE_PRINCIPAL'] = info['atividade_principal'][0]['text']
        data.loc[line,'MATRIZ'] = info['tipo']
        data.loc[line,'NATUREZA_JURIDICA'] = info['natureza_juridica']
        data.loc[line,'LOGRADOURO'] = info['logradouro'] 
        data.loc[line,'NUMERO'] = info['numero'] 
        data.loc[line,'COMPLEMENTO'] = info['complemento'] 
        data.loc[line,'BAIRRO'] = info['bairro'] 
        data.loc[line,'MUNICIPIO'] = info['municipio'] 
        data.loc[line,'UF'] = info['uf'] 
        data.loc[line,'CEP'] = info['cep'] 
        data.loc[line,'ENDEREÇO'] = get_company_adress(info)
    except:
        print(f"Erro: cnpj {cnpj} inválido")
    if line%10 == 0:
        print(f"Linha: {line} | CNPJs registrados: {len(info_dict)}")
    line += 1
    

data

Linha: 290 | CNPJs registrados: 5
Linha: 300 | CNPJs registrados: 14
Linha: 310 | CNPJs registrados: 19
Linha: 320 | CNPJs registrados: 23
Linha: 330 | CNPJs registrados: 25
Linha: 340 | CNPJs registrados: 25
Linha: 350 | CNPJs registrados: 32
Linha: 360 | CNPJs registrados: 41
Linha: 370 | CNPJs registrados: 46
Linha: 380 | CNPJs registrados: 49
Linha: 390 | CNPJs registrados: 55
Linha: 400 | CNPJs registrados: 61
Linha: 410 | CNPJs registrados: 64
Linha: 420 | CNPJs registrados: 70
Linha: 430 | CNPJs registrados: 77
Linha: 440 | CNPJs registrados: 78
Linha: 450 | CNPJs registrados: 84
Linha: 460 | CNPJs registrados: 87
Linha: 470 | CNPJs registrados: 91
Linha: 480 | CNPJs registrados: 94
Linha: 490 | CNPJs registrados: 95
Linha: 500 | CNPJs registrados: 95
Linha: 510 | CNPJs registrados: 95
Linha: 520 | CNPJs registrados: 95
Linha: 530 | CNPJs registrados: 95
Linha: 540 | CNPJs registrados: 95
Linha: 550 | CNPJs registrados: 99
Linha: 560 | CNPJs registrados: 105
Linha: 570 | CNPJs r

In [10]:
data.to_csv('Dados_processados_metade.csv')

In [ ]:
pd.read_parquet('Dados_processador_metade.parquet')

,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,ATIVIDADE_PRINCIPAL,MATRIZ,NATUREZA_JURIDICA,ENDEREÇO,LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,MUNICIPIO,UF,CEP
0,02/01/2003,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,"R$ 1.696,90",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,"Transporte rodoviário coletivo de passageiros,...",MATRIZ,206-2 - Sociedade Empresária Limitada,"R BARROS BARRETO, 74, RIO DE JANEIRO, RJ",R BARROS BARRETO,74,,BONSUCESSO,RIO DE JANEIRO,RJ,21.032-140
1,02/01/2003,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,"R$ 1.336,60",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,"Transporte rodoviário coletivo de passageiros,...",MATRIZ,206-2 - Sociedade Empresária Limitada,"R BARROS BARRETO, 74, RIO DE JANEIRO, RJ",R BARROS BARRETO,74,,BONSUCESSO,RIO DE JANEIRO,RJ,21.032-140
2,02/01/2003,***.868.251-**,31349202000177,JW TRANSPORTADORA TURÖSTICA,"R$ 1.396,43",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N,"Transporte rodoviário coletivo de passageiros,...",MATRIZ,206-2 - Sociedade Empresária Limitada,"R BARROS BARRETO, 74, RIO DE JANEIRO, RJ",R BARROS BARRETO,74,,BONSUCESSO,RIO DE JANEIRO,RJ,21.032-140
3,03/01/2003,***.004.131-**,00592717000170,FLORES ALVORADA,"R$ 8.585,00",C,OUTROS SERVIOS DE TERCEIROS-PESSOA JURÖDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,Comércio varejista de plantas e flores naturais,MATRIZ,206-2 - Sociedade Empresária Limitada,"SHC/SUL CL QUADRA 207 BLOCO B LOJA, 17, BRASIL...",SHC/SUL CL QUADRA 207 BLOCO B LOJA,17,,ASA SUL,BRASILIA,DF,70.253-520
4,03/01/2003,***.004.131-**,00839308000125,CRISTAL LIMPEZA - COMRCIO DE MATERIAIS PARA L...,"R$ 251,50",C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,Comércio varejista de produtos saneantes domis...,MATRIZ,206-2 - Sociedade Empresária Limitada,"ST SHCS CL QD 404 BL C LJS 09 E 13, SN, BRASIL...",ST SHCS CL QD 404 BL C LJS 09 E 13,SN,,ASA SUL,BRASILIA,DF,70.238-030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113337,04/12/2022,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIÇO,"518,07",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,None,,,,,,,
113338,04/12/2022,***.136.591-**,12550441000146,RESTAURANTE SOHO,"330,52",C,FORNECIMENTO DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,None,,,,,,,
113339,04/12/2022,***.136.591-**,03696869000100,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"54,66",C,GENEROS DE ALIMENTAÇO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,None,,,,,,,
113340,Fonte: SUPRIM,None,None,None,None,None,NaN,None,,,,None,,,,,,,
